In [ ]:
pip install --upgrade tensorflow==2.11.0

In [ ]:
import numpy as np
import datetime
import os
import random, shutil
import glob


import warnings
warnings.simplefilter('ignore')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib import pyplot
from matplotlib.image import imread

from os import makedirs,listdir
from shutil import copyfile
from random import seed
from random import random
import keras 
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D, Input
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from sklearn import metrics


sns.set_context('notebook')
sns.set_style('white')

In [ ]:
img_rows, img_cols = 150, 150
batch_size = 32
n_epochs = 10
n_classes = 2
val_split = 0.2
verbosity = 1
path = '/kaggle/input/waste-classification-data/DATASET/TRAIN/'
path_test = '/kaggle/input/waste-classification-data/DATASET/TEST'
input_shape = (img_rows, img_cols, 3) #RGB
labels = ['O', 'R']
seed = 10
checkpoint_path='ORnet.h5'

In [ ]:
train_datagen = ImageDataGenerator(
    validation_split = val_split,
    rescale=1.0/255.0,
	width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    validation_split = val_split,
    rescale=1.0/255.0,
	width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory = path,
    classes = labels,
    seed = seed,
    batch_size = batch_size, 
    class_mode='binary',
    shuffle = True,
    target_size=(img_rows, img_rows),
    subset = 'training'
)

In [ ]:
val_generator = val_datagen.flow_from_directory(
    directory = path,
    classes = labels,
    seed = seed,
    batch_size = batch_size, 
    class_mode='binary',
    shuffle = True,
    target_size=(img_rows, img_rows),
    subset = 'validation'
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory = path_test,
    classes = labels,
    class_mode='binary',
    seed = seed,
    batch_size = batch_size, 
    shuffle = True,
    target_size=(img_rows, img_rows)
)

In [ ]:
IMG_DIM = (100, 100)

train_files = glob.glob('/kaggle/input/waste-classification-data/DATASET/TRAIN/O/*')
train_imgs = [tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
train_labels = [fn.split('/')[6].split('.')[0].strip() for fn in train_files]

img_id = 0
O_generator = train_datagen.flow(train_imgs[img_id:img_id+1], train_labels[img_id:img_id+1],
                                   batch_size=1)
O = [next(O_generator) for i in range(0,5)]
fig, ax = plt.subplots(1,5, figsize=(16, 6))
print('Labels:', [item[1][0] for item in O])
l = [ax[i].imshow(O[i][0][0]) for i in range(0,5)]


In [ ]:
from keras.applications import vgg16
input_shape = (150, 150, 3)

vgg = vgg16.VGG16(include_top=False,
                        weights='imagenet',
                        input_shape=input_shape)



In [ ]:
output = vgg.layers[-1].output
output = tf.keras.layers.Flatten()(output)
basemodel = Model(vgg.input, output)

In [ ]:
basemodel.trainable = False
for layer in basemodel.layers: layer.trainable = False

In [ ]:
input_shape = basemodel.output_shape[1]

model = Sequential()
model.add(basemodel)
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(learning_rate=2e-5),
             metrics=['accuracy'])


In [ ]:
print(model.summary())

In [ ]:
from keras.callbacks import LearningRateScheduler
checkpoint_path='/kaggle/working/O_R_tlearn_image_augm_cnn_vgg16.h5'

# define step decay function
class LossHistory_(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(exp_decay(len(self.losses)))
        print('lr:', exp_decay(len(self.losses)))

def exp_decay(epoch):
    initial_lrate = 0.1
    k = 0.1
    lrate = initial_lrate * np.exp(-k*epoch)
    return lrate

# learning schedule callback
loss_history_ = LossHistory_()
lrate_ = LearningRateScheduler(exp_decay)

keras_callbacks = [
      EarlyStopping(monitor = 'loss', 
                    patience = 5, 
                    mode = 'min', 
                    min_delta=0.01),
      ModelCheckpoint(checkpoint_path, monitor='loss', save_best_only=True, mode='min')
]


callback_list_ = [loss_history_,lrate_,keras_callbacks]

In [ ]:
extract_feat_model = model.fit(train_generator,steps_per_epoch=10,epochs=50,
                              validation_data=val_generator,validation_steps=10,
                              verbose=1,callbacks=callback_list_)

In [ ]:
[layer.name for layer in basemodel.layers]

In [ ]:
basemodel.trainable = True
set_trainable = False

for layer in basemodel.layers:
    if layer.name in ['block4_conv1','block5_conv1']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model = Sequential()
model.add(basemodel)
model.add(Dense(512, activation='relu',input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

checkpoint_path = '/kaggle/working/O_R_tlearn_image_augm_fine_tune_cnn_vgg16.h5'

loss_history_ = LossHistory_()
lrate = LearningRateScheduler(exp_decay)

keras_callbacks = [
    EarlyStopping(monitor='loss',patience=5,mode='min',min_delta=0.01),
    ModelCheckpoint(checkpoint_path,monitor='loss',save_best_only=True,mode='min')
]

callback_list = [loss_history_,lrate,keras_callbacks]


In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(learning_rate=1e-5),
             metrics=['accuracy'])

fine_tune_model = model.fit(train_generator,steps_per_epoch=10,epochs=50,
                           callbacks=callback_list,validation_data=val_generator,
                           validation_steps=10,verbose=1)

In [ ]:
extract_feat_model = tf.keras.models.load_model('/kaggle/working/O_R_tlearn_image_augm_cnn_vgg16.h5')
fine_tune_model = tf.keras.models.load_model('/kaggle/working/O_R_tlearn_image_augm_fine_tune_cnn_vgg16.h5')


In [ ]:
from sklearn.utils import shuffle

IMG_DIM = (150,150)

#Read in all O and R test images file path and shuffle and take 50 images
test_files_O = glob.glob('/kaggle/input/waste-classification-data/DATASET/TEST/O/*')
test_files_R = glob.glob('/kaggle/input/waste-classification-data/DATASET/TEST/R/*')
test_files = test_files_O + test_files_R
test_files = shuffle(test_files)[0:50]

#extract label from file path
test_imgs = [tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(img,target_size=IMG_DIM)) for img in test_files]
test_imgs = np.array(test_imgs)
test_labels = [fn.split('/')[6].split('.')[0].strip() for fn in test_files]

#standardize
test_img_scale = test_imgs.astype('float32')
test_img_scale /= 255

In [ ]:
class2_num_it = lambda l: [0 if x=='O' else 1 for x in l]
num2_class_it = lambda l: ['O' if x<0.5 else 'R' for x in l]
test_labels_enc = class2_num_it(test_labels)

In [ ]:
predictions_extract_feat_model = extract_feat_model.predict(test_img_scale, verbose=0)
predictions_fine_tune_model = fine_tune_model.predict(test_img_scale, verbose=0)

In [ ]:
predictions_extract_feat_model = num2_class_it(predictions_extract_feat_model)
predictions_fine_tune_model = num2_class_it(predictions_fine_tune_model)

In [ ]:
print('Extract Features Model')
print(metrics.classification_report(test_labels, predictions_extract_feat_model))
print('Fine-Tuned Model')
print(metrics.classification_report(test_labels, predictions_fine_tune_model))

In [ ]:
custom_im = test_img_scale[2]
plt.imshow(custom_im)

num2_class_it(extract_feat_model.predict(custom_im.reshape((1,test_img_scale.shape[1],
                                                           test_img_scale.shape[2],
                                                           test_img_scale.shape[3])),verbose=0))

In [ ]:
custom_im = test_img_scale[3]
plt.imshow(custom_im)

num2_class_it(extract_feat_model.predict(custom_im.reshape((1,test_img_scale.shape[1],
                                                          test_img_scale.shape[2],
                                                          test_img_scale.shape[3])),verbose=0))